In [1]:
import pandas as pd
import numpy as np

In [2]:
cols = ['j', 'LH']
c1 = np.array([i for i in range(1,6)])
c2 = np.array([10,20,40,60,120])
LH_df = pd.DataFrame({'j': c1, 'LH': c2})
LH_df.set_index('LH')
LH_df

,j,LH
0,1,10
1,2,20
2,3,40
3,4,60
4,5,120


In [3]:
col = ['Risk Factor', 'LH']
c3 = np.array(['IR','CrSprd', 'EQ', 'FX','COMMO'])
c4 = np.array([10,20,10,10,120])
RF_df = pd.DataFrame({'Risk_Factor': c3, 'LH': c4})
# RF_df = RF_df.set_index('Risk Factor');
RF_df

,Risk_Factor,LH
0,IR,10
1,CrSprd,20
2,EQ,10
3,FX,10
4,COMMO,120


In [4]:
combined_df = pd.merge(RF_df, LH_df, on='LH', how='outer');
combined_df

,Risk_Factor,LH,j
0,IR,10,1
1,EQ,10,1
2,FX,10,1
3,CrSprd,20,2
4,COMMO,120,5
5,NaN,40,3
6,NaN,60,4


In [5]:
def cVar(cnt):
    return pnl_df.iloc[cnt].where(pnl_df.iloc[cnt] < pnl_df.iloc[cnt][-1]).dropna().mean()/0.025

def LH(cnt):
#     if np.sum(pnl_df.iloc[cnt])!=0:
        # get risk factor
    risk_factor = pnl_df.index.get_level_values(1)[cnt]
    j = combined_df.loc[combined_df.Risk_Factor == risk_factor].iloc[0][2]
    if j <= 1:
        LHj = 0
        LHk = 0
    else:
        LHj = combined_df.loc[combined_df.Risk_Factor == risk_factor].iloc[0][1]
        j -= 1
        LHk = LH_df.loc[LH_df.j == j].iloc[0][1]
    return LHj, LHk

In [16]:
# '/Users/lloydzhang/Desktop/FRTB/PNL_10D.xlsx'
def ES(PNL):
    global pnl_df
    df = pd.read_excel(PNL)
    scenarioDATES = df.iloc[0][13].split(' ') # column position might change
    pnl_df =  pd.DataFrame(columns = scenarioDATES)
    for i in range(80): #length might vary in different files
        pnl_str = np.array(df.iloc[:,23].dropna())[i].split(' ')
        pnl_list = [float(i) for i in pnl_str]
        pnl_df.loc[i]= pnl_list
    pnl_df = pnl_df.diff(axis=1)
    tradetype = []
    riskFactor = []
    for i in range(20): # relative to the number of total rows
        tradetype_cnt = 6 + i*11
        riskFactor_cnt = i*11
        tradetype.append(df.iloc[tradetype_cnt][20])
        riskFactor.append(df.iloc[riskFactor_cnt][11].split(' ')[0])
        riskFactor.append(df.iloc[riskFactor_cnt][11].split(' ')[1])
        riskFactor.append(df.iloc[riskFactor_cnt][11].split(' ')[2])
        riskFactor.append(df.iloc[riskFactor_cnt][11].split(' ')[3])
    pnl_df['Risk Factor'] = riskFactor
    pnl_df['Trade Type'] = np.repeat(tradetype, 4)
    pnl_df = pnl_df.set_index(['Trade Type','Risk Factor'])
    pnl_df['VaR'] = pnl_df.quantile(0.025,axis=1)
    ES = np.zeros(80)
    for i in range(20):
        T_cnt = 11*i 
        T = df.iloc[T_cnt][9]

        p0_cnt = 4*i
        ES_p0 = cVar(p0_cnt)
        ES[p0_cnt] = ES_p0
        sqr0 = np.nan_to_num((ES_p0*np.sqrt((LH(p0_cnt)[0] - LH(p0_cnt)[1])/T))**2)

        p1_cnt = 1 + 4*i
        ES_p1 = cVar(p1_cnt)
        ES[p1_cnt] = ES_p1
        sqr1 = np.nan_to_num((ES_p1*np.sqrt((LH(p1_cnt)[0] - LH(p1_cnt)[1])/T))**2)

        p2_cnt = 2 + 4*i
        ES_p2 = cVar(p2_cnt)
        ES[p2_cnt] = cVar(p2_cnt)
        sqr2 = np.nan_to_num((ES_p2*np.sqrt((LH(p2_cnt)[0] - LH(p2_cnt)[1])/T))**2)

        pt_cnt = 3 + 4*i
        ES_p_total = cVar(pt_cnt)

        ES[pt_cnt] = np.sqrt(ES_p_total**2+(sqr0 + sqr1 + sqr2))
    pnl_df['ES'] = ES
    
    return pnl_df.iloc[:,-1]

In [17]:
ES('/Users/lloydzhang/Desktop/FRTB/HW3/PNL_10D.xlsx')

Trade Type  Risk Factor
SWAP        IR            -7.191116e+07
            FX                      NaN
            EQ                      NaN
            TOTAL          7.191116e+07
            IR            -7.191294e+07
            FX                      NaN
            EQ                      NaN
            TOTAL          7.191294e+07
            IR            -9.699479e+07
            FX                      NaN
            EQ                      NaN
            TOTAL          9.699479e+07
            IR            -2.285012e+08
            FX                      NaN
            EQ                      NaN
            TOTAL          2.285012e+08
            IR            -1.873118e+07
            FX                      NaN
            EQ                      NaN
            TOTAL          1.873118e+07
            IR            -2.613646e+08
            FX                      NaN
            EQ                      NaN
            TOTAL          2.613646e+08
FUT         IR  